<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `GRHayL`: GRMHD Source Terms

## Author: Terrence Pierre Jacques

<a id='intro'></a>

**Notebook Status:** <font color=green><b> Validated </b></font>

**Validation Notes:** This code produces the expected C code for the generated functions.

## This module presents the functionality of [IGM_All_Source_Terms.py](/edit/source_terms/IGM_All_Source_Terms.py).

## Introduction: 
This notebook documents and self-validates the C code generation of the source terms for our GRMHD formulation, generated NRPy+ to be used later by GRHayL. We call on the functions defined in [GRMHD_equations_new_version.py](/edit/GRMHD_formulation/GRMHD_equations_new_version.py), documented in [Tutorial-GRMHD_Equations-Cartesian.ipynb](../GRMHD_formulation/Tutorial-GRMHD_Equations-Cartesian.ipynb).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#prelim): Preliminaries
1. [Step 2](#source_terms): GRMHD Source Terms
1. [Step 3](#C_code): C Code Generation
    1. [Step 3.a](#C_code_a): Accessing Structs
    1. [Step 3.b](#C_code_b): Printing the C Function
1. [Step 4](#code_validation): Code Validation against C code generated by pyhton module
1. [Step 5](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Step 1. Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

Here we import some core NRPy+ and python modules.

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
GRMHD_dir_path = os.path.join("../GRMHD_formulation/")
sys.path.append(GRMHD_dir_path)
import GRMHD_equations_new_version as GRMHD    # NRPy+: Generate general relativistic magnetohydrodynamics equations

nrpy_dir_path = os.path.join("../nrpy/")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 1: The StildeD RHS *source* term
# Step P1: Import needed NRPy+ core modules:
from outputC import outputC, outCfunction # NRPy+: Core C code output module
import NRPy_param_funcs as par        # NRPy+: Parameter interface
import reference_metric as rfm        # NRPy+: Reference metric support
import cmdline_helper as cmd          # NRPy+: Multi-platform Python command-line interface
import sympy as sp                # SymPy: The Python computer algebra package upon which NRPy+ depends

thismodule = "IGM_Source_Terms"

Ccodesdir = "IGM_standalone_Ccodes/"
cmd.mkdir(os.path.join(Ccodesdir))

par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

CoordSystem = "Cartesian"

# Set coordinate system to dst_basis
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

import IGM_All_Source_Terms as ST

<a id='source_terms'></a>

# Step 2: GRMHD Source Terms \[Back to [top](#toc)\]
$$\label{stilde_source}$$

Here we call on functions within [GRMHD_equations_new_version.py](/edit/GRMHD_formulation/GRMHD_equations_new_version.py) to define the source terms for the $\tilde{\tau}$ and $\tilde{S}_i$ evolution equations,

\begin{align}
\tilde{\tau} &\propto \alpha \sqrt{\gamma} \left[\left(T^{00}\beta^i\beta^j + 2 T^{0i}\beta^j + T^{ij} \right) K_{ij}
- \left(T^{00}\beta^i + T^{0i} \right) \partial_i\alpha \right], \\
\tilde{S}_i &\propto \frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu} g_{\mu\nu,i},
\end{align}

and generate the C code using the `outputC` python module. Note that while we define the formalism to be BSSN here, ADM may also be used.

In [2]:
# Generate SymPy symbolic expressions
formalism="ADM"
GRMHD.set_up_base_vars(formalism)

GRMHD.compute_vU_from_u4U__no_speed_limit(GRMHD.u4U)   

GRMHD.compute_sqrtgammaDET(GRMHD.gammaDD)
GRMHD.compute_smallb4U(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.u4U, GRMHD.BU, GRMHD.sqrt4pi)
GRMHD.compute_smallbsquared(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.smallb4U)

# First compute stress-energy tensor T4UU and T4UD:
GRMHD.compute_T4UU(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.rho_b,GRMHD.P,GRMHD.h,GRMHD.u4U, GRMHD.smallb4U, GRMHD.smallbsquared)
GRMHD.compute_T4UD(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.T4UU)

# Compute conservative variables in terms of primitive variables
GRMHD.compute_rho_star(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.rho_b,GRMHD.u4U)
GRMHD.compute_tau_tilde(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UU,GRMHD.rho_star)
GRMHD.compute_S_tildeD(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

# Next compute fluxes of conservative variables
GRMHD.compute_rho_star_fluxU(GRMHD.VU,GRMHD.rho_star)
GRMHD.compute_tau_tilde_fluxU(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.VU, GRMHD.T4UU, GRMHD.rho_star)
GRMHD.compute_S_tilde_fluxUD (GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

# Then declare derivatives & compute g4DD_zerotimederiv_dD
GRMHD.compute_g4DD_zerotimederiv_dD(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.gammaDD_dD,GRMHD.betaU_dD,GRMHD.alpha_dD)

# Then compute source terms on tau_tilde and S_tilde equations
GRMHD.compute_tau_tilde_source_term(GRMHD.KDD,GRMHD.betaU,GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.alpha_dD, GRMHD.T4UU)
GRMHD.compute_S_tilde_source_termD(GRMHD.alpha,GRMHD.sqrtgammaDET,GRMHD.g4DD_zerotimederiv_dD, GRMHD.T4UU)


<a id='C_code'></a>

# Step 3: C Code Generation \[Back to [top](#toc)\]
$$\label{C_code}$$

In this section we now generate the C code to do these source terms calculations.


<a id='C_code_a'></a>

## Step 3.a: Accessing Structs \[Back to [top](#toc)\]
$$\label{C_code_a}$$

Because each gem within GRHayL is meant to be used in a point-wise fashion, we store values within structs and access them within a given C function. The below simply prints the C code necessary to access data within the relevant structs, which are __reconstructed_prims__, __metric_quantities__, __metric_quantities_derivatives__.

In [3]:
# GRHayL structs:

# typedef struct primitive_quantities {
# double rho, press, eps;
# double vx, vy, vz;
# double Bx, By, Bz;
# double entropy, Y_e, temperature;
# double enthalpy;
# } primitive_quantities;

# typedef struct conservative_quantities {
#   double rho, tau, Y_e;
#   double S_x, S_y, S_z;
#   double entropy;
# } conservative_quantities;

# typedef struct metric_quantities {
#   double adm_gxx, adm_gxy, adm_gxz;
#   double adm_gyy, adm_gyz, adm_gzz;
#   double adm_gupxx, adm_gupxy, adm_gupxz;
#   double adm_gupyy, adm_gupyz, adm_gupzz;
#   double betax, betay, betaz;
#   double lapse, lapseinv;
#   double psi2, psi4, psi6;
#   double psi4inv, lapseinv2;
#   double g4dn[4][4],g4up[4][4];
# } metric_quantities;

# typedef struct metric_derivatives {
#   double lapse[3];
#   double betax[3];
#   double betay[3];
#   double betaz[3];
#   double adm_gxx[3];
#   double adm_gxy[3];
#   double adm_gxz[3];
#   double adm_gyy[3];
#   double adm_gyz[3];
#   double adm_gzz[3];
# } metric_derivatives;

In [4]:
tau_tilde_source_term_free_symbols = GRMHD.tau_tilde_source_term.free_symbols
S_tilde_source_termD0_free_symbols = GRMHD.S_tilde_source_termD[0].free_symbols
S_tilde_source_termD1_free_symbols = GRMHD.S_tilde_source_termD[1].free_symbols
S_tilde_source_termD2_free_symbols = GRMHD.S_tilde_source_termD[2].free_symbols

all_free_sysmbols = tau_tilde_source_term_free_symbols.union(\
                             S_tilde_source_termD0_free_symbols, 
                             S_tilde_source_termD1_free_symbols,
                             S_tilde_source_termD2_free_symbols)

prims_NRPy = ["u4U0", "u4U1", "u4U2", "u4U3", "BU0", "BU1", "BU2", "P", "rhob"]
prims_GRHayL = ["ut", "vx*u4U0", "vy*u4U0", "vz*u4U0", "Bx", "By", "Bz", "press", "rho"]

prestring = r"""
double h, cs2;

eos->compute_h_and_cs2(prims, &h, &cs2);
"""

for i in range(len(prims_NRPy)):
    prestring += "const double "+prims_NRPy[i]+" = prims->"+prims_GRHayL[i]+";\n"
    
prestring += "const double "+str(GRMHD.alpha)+" = metric->lapse;\n"

checker = []

if formalism=="BSSN":
    # BSSN quantites
    prestring += "const double "+str(GRMHD.Bq.trK)+" = metric_quantities->"+str(GRMHD.Bq.trK)+";\n"
    prestring += "const double "+str(GRMHD.Bq.cf)+" = metric_quantities->"+str(GRMHD.Bq.cf)+";\n"

    for i in range(3):
        vetU_var = GRMHD.Bq.vetU[i]
        prestring += "const double "+str(vetU_var)+" = metric_quantities->"+str(vetU_var)+";\n"

    for i in range(3):
        for j in range(3):
            aDD_var = GRMHD.Bq.aDD[i][j]
            if aDD_var in checker: 
                continue
            prestring += "const double "+str(aDD_var)+" = metric_quantities->"+str(aDD_var)+";\n"
            checker.append(aDD_var)

    for i in range(3):
        for j in range(3):
            hDD_var = GRMHD.Bq.hDD[i][j]
            if hDD_var in checker: 
                continue
            prestring += "const double "+str(hDD_var)+" = metric_quantities->"+str(hDD_var)+";\n"
            checker.append(hDD_var)

    for var in all_free_sysmbols:
        if "_dD" in str(var):
            prestring += "const double "+str(var)+" = metric_quantities_derivatives->"+str(var)+";\n"

else:
    #ADM quantities
#     for i in range(3):
#             betaU_var = GRMHD.betaU[i]
#             prestring += "const double "+str(betaU_var)+" = metric_quantities->"+str(betaU_var)+";\n"

#     for i in range(3):
#         for j in range(3):
#             KDD_var = GRMHD.KDD[i][j]
#             if KDD_var in checker: 
#                 continue
#             prestring += "const double "+str(KDD_var)+" = metric_quantities->"+str(KDD_var)+";\n"
#             checker.append(KDD_var)

#     for i in range(3):
#         for j in range(3):
#             gammaDD_var = GRMHD.gammaDD[i][j]
#             if gammaDD_var in checker: 
#                 continue                
#             prestring += "const double "+str(gammaDD_var)+" = metric_quantities->"+str(gammaDD_var)+";\n"
#             checker.append(gammaDD_var)

#     for var in all_free_sysmbols:
#         if "_dD" in str(var):
#             prestring += "const double "+str(var)+" = metric_quantities_derivatives->"+str(var)+";\n"
    
    
    prestring += "const double betaU0 = metric->betax;\n"
    prestring += "const double betaU1 = metric->betay;\n"
    prestring += "const double betaU2 = metric->betaz;\n"


    prestring += "const double gammaDD00 = metric->adm_gxx;\n"
    prestring += "const double gammaDD01 = metric->adm_gxy;\n"
    prestring += "const double gammaDD02 = metric->adm_gxz;\n"

    prestring += "const double gammaDD11 = metric->adm_gyy;\n"
    prestring += "const double gammaDD12 = metric->adm_gyz;\n"

    prestring += "const double gammaDD22 = metric->adm_gzz;\n"

    prestring += "const double KDD00 = curv->Kxx;\n"
    prestring += "const double KDD01 = curv->Kxy;\n"
    prestring += "const double KDD02 = curv->Kxz;\n"

    prestring += "const double KDD11 = curv->Kyy;\n"
    prestring += "const double KDD12 = curv->Kyz;\n"

    prestring += "const double KDD22 = curv->Kzz;\n"


    prestring += "const double alpha_dD0 = metric_derivs->lapse[0];\n"
    prestring += "const double alpha_dD1 = metric_derivs->lapse[1];\n"
    prestring += "const double alpha_dD2 = metric_derivs->lapse[2];\n"

    for i in range(3):
        prestring += f"const double betaU_dD0{i} = metric_derivs->betax[{i}];\n"
        prestring += f"const double betaU_dD1{i} = metric_derivs->betay[{i}];\n"
        prestring += f"const double betaU_dD2{i} = metric_derivs->betaz[{i}];\n"

        prestring += f"const double gammaDD_dD00{i} = metric_derivs->adm_gxx[{i}];\n"
        prestring += f"const double gammaDD_dD01{i} = metric_derivs->adm_gxy[{i}];\n"
        prestring += f"const double gammaDD_dD02{i} = metric_derivs->adm_gxz[{i}];\n"

        prestring += f"const double gammaDD_dD11{i} = metric_derivs->adm_gyy[{i}];\n"
        prestring += f"const double gammaDD_dD12{i} = metric_derivs->adm_gyz[{i}];\n"

        prestring += f"const double gammaDD_dD22{i} = metric_derivs->adm_gzz[{i}];\n"


<a id='C_code_b'></a>

## Step 3.b: Printing the C Function \[Back to [top](#toc)\]
$$\label{C_code_b}$$

Here we now print out the C function that will be used in a point-wise fashion to calculate the source terms.

In [5]:
outCparams = "outCverbose=False,CSE_sorting=canonical,CSE_enable=True"

desc = "Adds source term and connection terms to Stilde and tau_tilde"
includes = ["NRPy_basic_defines.h"]
name = "calculate_all_source_terms"
vars_to_write = ["cons->S_x", "cons->S_y", "cons->S_z", "cons->tau"]

vars_rhs = [GRMHD.S_tilde_source_termD[0], 
            GRMHD.S_tilde_source_termD[1], 
            GRMHD.S_tilde_source_termD[2], 
            GRMHD.tau_tilde_source_term]

body = outputC(vars_rhs, vars_to_write, params=outCparams, 
           filename="returnstring", prestring=prestring)

c_type = "void"

params  = "const primitive_quantities *restrict prims, "
params  += "const eos_parameters *restrict eos, "
params  += "const metric_quantities *restrict metric, "
params  += "const extrinsic_curvature *restrict curv, "
params  += "const metric_derivatives *restrict metric_derivs, "
params  += "conservative_quantities *restrict cons"

outCfunction(
    outfile=os.path.join(Ccodesdir,name+".c"),
    includes=includes,
    desc=desc,
    c_type=c_type, name=name, params=params,
    enableCparameters=False,
    body=body)

Output C function calculate_all_source_terms() to file IGM_standalone_Ccodes/calculate_all_source_terms.c


<a id='code_validation'></a>

# Step 4:  Code Validation against `IGM_All_Source_Terms` Module \[Back to [top](#toc)\]
$$\label{code_validation}$$


Here, as a code validation check, we verify agreement in the C code generated by 
1. this tutorial and 
2. the [IGM_All_Source_Terms.py](/edit/source_terms/IGM_All_Source_Terms.py) module.

In [6]:
# Define the directory that we wish to validate against:
valdir = "IGM_val/"
cmd.mkdir(os.path.join(valdir))

import IGM_All_Source_Terms as ST
ST.Cfunction__GRMHD_SourceTerms(valdir, includes=includes, formalism=formalism)

import difflib
import sys

print("Printing difference between original C code and this code...")
# Open the files to compare
files = ["calculate_all_source_terms.c"]

for file in files:
    print("Checking file " + file)
    with open(os.path.join(valdir,file)) as file1, open(os.path.join(Ccodesdir,file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(Ccodesdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .c file. See differences above.")
            sys.exit(1)

Output C function calculate_all_source_terms() to file IGM_val/calculate_all_source_terms.c
Printing difference between original C code and this code...
Checking file calculate_all_source_terms.c
No difference. TEST PASSED!


<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GRHayL-All_Source_Terms_NRPY_Gen](Tutorial-GRHayL-All_Source_Terms_NRPY_Gen.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [7]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GRHayL-All_Source_Terms_NRPY_Gen")

Traceback (most recent call last):
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/traitlets/traitlets.py", line 537, in get
    value = obj._trait_values[self.name]
KeyError: 'template_paths'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/terrence/environments/ve3.10/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/jupyter_core/application.py", line 264, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/nbconvert/nbconvertapp.py", line 369, in start
    self.convert_notebooks()
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packag